In [2]:
import re
import json
import pandas as pd
from urllib.request import urlopen
from os import path

In [3]:
def read_page(url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    return html    

def read_sitemap(url, start):
    sitemap = read_page(url)    
    regex = '<loc>'+start+'([^<]*)</loc>'
    matches = re.findall(regex, sitemap)
    keys = [match for match in matches]
    keys.sort()
    return keys

def get_delta(keys, pickle):
    if not path.exists(pickle):
        return keys
    df = pd.read_pickle(pickle)
    done = df[df['status']==3]['estateId'].values
    return [key for key in keys if key not in done]

def download_pages(keys, start, folder):
    for key in keys:
        try:
            s = read_page(start+key)
            begin = '<script id="__NEXT_DATA__" type="application/json">'
            end = '</script>'
            i = s.find(begin)
            if i > 0:
                j = s.find(end, i)
                ss = s[(i+len(begin)):j]
                data = json.loads(ss)
                data = data['props']['initialProps']['pageProps']['estate']
                with open('{}/{}.json'.format(folder, key), 'w', encoding='utf-8') as f:
                    f.write(json.dumps(data))
            else:
                print(key)
        except Exception as e:
            print(key, str(e))

In [4]:
# download new entries and reload not sold estate

keys = read_sitemap('https://www.krogsveen.no/sitemap.xml','https://www.krogsveen.no/kjope/boliger-til-salgs/')
keys = get_delta(keys, 'assets/Krogsveen.pkl')
print(len(keys))

download_pages(keys, 'https://www.krogsveen.no/kjope/boliger-til-salgs/', 'data')

1145
